In [3]:
!pip install sqlalchemy pyodbc --quiet

In [5]:
#importing 
import pandas as pd
from sqlalchemy import create_engine
import urllib
import os

csv_path = r"C:\Users\TANYA\Downloads\housing_data.csv"

print("CSV exists:", os.path.exists(csv_path))
df = pd.read_csv(csv_path)
print("Shape:", df.shape)
df.head(3)


CSV exists: True
Shape: (1460, 81)


,Unnamed: 0,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,SC60,RL,65,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,No,No,No,0,Feb,2008,WD,Normal,208500
1,1,SC20,RL,80,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,No,No,No,0,May,2007,WD,Normal,181500
2,2,SC60,RL,68,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,No,No,No,0,Sep,2008,WD,Normal,223500


In [6]:
#Experience and satisfaction score
df['Satisfaction_Score'] = (df['OverallQual'] + df['OverallCond']) / 2
df['Experience_Score'] = (2025 - df['YearBuilt']) / 10
df[['OverallQual', 'OverallCond', 'YearBuilt', 'Satisfaction_Score', 'Experience_Score']].head()

,OverallQual,OverallCond,YearBuilt,Satisfaction_Score,Experience_Score
0,7,5,2003,6.0,2.2
1,6,8,1976,7.0,4.9
2,7,5,2001,6.0,2.4
3,7,5,1915,6.0,11.0
4,8,5,2000,6.5,2.5


In [17]:
#creating SQLite connection
from sqlalchemy import create_engine

engine = create_engine("sqlite:///tanya_housing.db")
print("SQLite connection created successfully!")

SQLite connection created successfully!


In [16]:
#Pushing dataset into the SQLite database
df.to_sql('housing_table_tanya', con=engine, if_exists='replace', index=False)
print("Data pushed to SQLite database successfully!")

Data pushed to SQLite database successfully!


In [15]:
#verifying my data
check = pd.read_sql('SELECT SalePrice, Satisfaction_Score, Experience_Score FROM housing_table_tanya LIMIT 5', con=engine)
check

,SalePrice,Satisfaction_Score,Experience_Score
0,208500,6.0,2.2
1,181500,7.0,4.9
2,223500,6.0,2.4
3,140000,6.0,11.0
4,250000,6.5,2.5


In [ ]:
In this project, I connected my Jupyter Notebook with a database engine using SQLAlchemy.
I demonstrated data export by pushing the housing dataset into an SQLite database,
including new analytical columns for satisfaction and experience scores

 Verification steps (data successfully pushed and read from SQL)


In [27]:
import os
print("CSV exists:", os.path.exists(r"C:\Users\TANYA\Downloads\housing_data.csv"))
print("Notebook working directory:", os.getcwd())

try:
    print("df shape:", df.shape)
    display(df.head(3))
except NameError:
    print("df is not defined — run the cell that does pd.read_csv(...)")

CSV exists: True
Notebook working directory: C:\Users\TANYA
df shape: (1460, 83)


,Unnamed: 0,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Satisfaction_Score,Experience_Score
0,0,SC60,RL,65,8450,Pave,NaN,Reg,Lvl,AllPub,...,No,No,0,Feb,2008,WD,Normal,208500,6.0,2.2
1,1,SC20,RL,80,9600,Pave,NaN,Reg,Lvl,AllPub,...,No,No,0,May,2007,WD,Normal,181500,7.0,4.9
2,2,SC60,RL,68,11250,Pave,NaN,IR1,Lvl,AllPub,...,No,No,0,Sep,2008,WD,Normal,223500,6.0,2.4


In [26]:
cols = ['Satisfaction_Score','Experience_Score']
exist = [c in df.columns for c in cols]
print(dict(zip(cols, exist)))
if all(exist):
    display(df[cols].head())
else:
    print("Missing columns — run the cell that creates them (I provided code earlier).")

{'Satisfaction_Score': True, 'Experience_Score': True}


,Satisfaction_Score,Experience_Score
0,6.0,2.2
1,7.0,4.9
2,6.0,2.4
3,6.0,11.0
4,6.5,2.5


In [23]:
from sqlalchemy import create_engine, inspect
import os

db_file = "tanya_housing.db"
print("DB file exists in cwd?:", os.path.exists(db_file))

try:
    engine  # exists?
    insp = inspect(engine)
    print("Tables in DB:", insp.get_table_names())
except NameError:
    print("No 'engine' variable found — run the cell that does create_engine('sqlite:///tanya_housing.db')")
except Exception as e:
    print("Engine exists but failed to inspect:", e)

DB file exists in cwd?: True
Tables in DB: ['housing_table_tanya']


In [24]:
import pandas as pd
try:
    q = "SELECT SalePrice, Satisfaction_Score, Experience_Score FROM housing_table_tanya LIMIT 5"
    sample_back = pd.read_sql(q, con=engine)
    display(sample_back)
except Exception as e:
    print("Error reading table (maybe table name different):", e)
    try:
        print("Available tables:", inspect(engine).get_table_names())
    except:
        pass

,SalePrice,Satisfaction_Score,Experience_Score
0,208500,6.0,2.2
1,181500,7.0,4.9
2,223500,6.0,2.4
3,140000,6.0,11.0
4,250000,6.5,2.5


In [25]:
# counts and nulls for the new columns
print("Rows:", len(df))
print("Satisfaction nulls:", df['Satisfaction_Score'].isna().sum())
print("Experience nulls:", df['Experience_Score'].isna().sum())

# check a few medians
print("Median SalePrice:", df['SalePrice'].median())
print("Median Satisfaction_Score:", df['Satisfaction_Score'].median())

Rows: 1460
Satisfaction nulls: 0
Experience nulls: 0
Median SalePrice: 163000.0
Median Satisfaction_Score: 6.0
